# Imports

In [1]:
from eth_account import Account
from x402.clients.requests import x402_requests
import json
import time
import os
from dotenv import load_dotenv
import pandas as pd
from chartengineer import ChartMaker
from typing import Optional
import sys

try:
    import websockets
    import aiohttp
except ImportError as e:
    print(f"Missing: pip install websockets aiohttp")
    sys.exit(1)

load_dotenv()

True

# Environment Variables + Client

In [2]:
PRIVATE_KEY = os.getenv("PRIVATE_KEY")
WALLET_ADDRESS = os.getenv("WALLET_ADDRESS", "0x27238ff1Bf4Ef450AF1214a95963B9a92b351f08")
NETWORK = os.getenv("NETWORK", "opt-mainnet")
BASE_URL = os.getenv("BASE_URL", "https://r3467d7khd8b94sfguhrr273lo.ingress.akashprovid.com")
# BASE_URL = "http://localhost:8015"

account = Account.from_key(PRIVATE_KEY)
session = x402_requests(account)

# Utils

In [3]:
class X402WebSocketClient:
    """WebSocket client with x402 payment at handshake."""

    def __init__(
        self,
        base_url: str,
        wallet_address: str,
        private_key: str,
        network: str = "eth-mainnet",
    ):
        self.base_url = base_url.rstrip("/")
        self.wallet_address = wallet_address
        self.network = network
        self.ws = None
        self.session_info = None

        # Initialize x402 client
        self.account = Account.from_key(private_key)
        self.x402_session = x402_requests(self.account)
        print(f"[x402] Account: {self.account.address}")

    def _ws_url(self) -> str:
        """Build WebSocket URL."""
        base = self.base_url.replace("http://", "ws://").replace("https://", "wss://")
        return f"{base}/wallet/live?address={self.wallet_address}&network={self.network}"

    def _http_url(self) -> str:
        """HTTP URL for payment."""
        return f"{self.base_url}/wallet/live?address={self.wallet_address}&network={self.network}"

    async def connect(self) -> None:
        """
        Connect using x402 payment.

        Flow:
        1. Use x402_requests to make HTTP request (handles 402 payment)
        2. The payment receipt from successful request is reused for WebSocket
        """
        print("1. Making x402 payment via HTTP...")
        url = self._http_url()

        # x402_requests handles: request -> 402 -> pay -> retry with receipt
        response = self.x402_session.get(url)

        if response.status_code != 200:
            raise Exception(f"Payment failed: {response.status_code}")

        print("   ✓ Payment successful")

        # Get the receipt that was used for the successful request
        receipt = response.request.headers.get("X-PAYMENT", "")

        if not receipt:
            print("   ! No receipt found, trying WebSocket anyway...")

        print("2. Connecting to WebSocket...")
        ws_url = self._ws_url()

        # For WebSocket, we need to handle 402 differently since websockets
        # library doesn't support x402 directly. We'll pass the receipt as header.
        try:
            headers = {"X-PAYMENT": receipt} if receipt else {}
            self.ws = await websockets.connect(ws_url, additional_headers=headers)
        except websockets.exceptions.InvalidStatusCode as e:
            if e.status_code == 402:
                print("   ! WebSocket got 402, making separate payment...")
                # Make another payment specifically for WS
                response2 = self.x402_session.get(url)
                receipt2 = response2.request.headers.get("X-PAYMENT", "")
                headers = {"X-PAYMENT": receipt2} if receipt2 else {}
                self.ws = await websockets.connect(ws_url, additional_headers=headers)
            else:
                raise

        # Wait for connected message
        msg = await self.ws.recv()
        data = json.loads(msg)

        if data.get("type") == "error":
            raise Exception(f"Connection error: {data.get('message')}")

        if data.get("type") != "connected":
            raise Exception(f"Unexpected: {data}")

        self.session_info = data
        print("3. Connected!")
        print(f"   Session: {data.get('session_id')}")
        print(f"   Expires: {data.get('expires_at')}")
        print()

    async def stream(self) -> None:
        """Stream portfolio updates."""
        if not self.ws:
            raise Exception("Not connected")

        print("Streaming portfolio updates...")
        print("-" * 60)

        update_count = 0

        try:
            async for msg in self.ws:
                data = json.loads(msg)
                msg_type = data.get("type")

                if msg_type == "portfolio_update":
                    update_count += 1
                    self._print_update(data, update_count)

                elif msg_type == "session_ending":
                    print(f"\n⚠️  Session ending in {data.get('seconds_remaining')}s")

                elif msg_type == "error":
                    if data.get("code") == "session_expired":
                        print("\n⏰ Session expired")
                        break
                    print(f"\n❌ Error: {data.get('message')}")

        except websockets.ConnectionClosed as e:
            print(f"\n🔌 Connection closed: {e.code} {e.reason}")

        print(f"\nSession complete: {update_count} updates")

    def _print_update(self, data: dict, count: int) -> None:
        """Pretty print portfolio update."""
        ts = data.get("timestamp", "")[:19]
        value = data.get("total_value_usd", 0)
        change = data.get("change_24h")
        positions = data.get("positions", [])

        change_str = ""
        if change is not None:
            arrow = "📈" if change >= 0 else "📉"
            change_str = f" {arrow} {change * 100:+.2f}%"

        print(f"[{ts}] #{count} 💰 ${value:,.2f}{change_str}")

        # Show all positions on every update
        for p in positions:
            print(f"   {p['symbol']}: ${p['value_usd']:,.2f} ({p['weight']*100:.1f}%)")

    async def disconnect(self) -> None:
        if self.ws:
            await self.ws.close()
            self.ws = None


In [4]:
def describe_df(df: pd.DataFrame):
    print("DataFrame Description:")
    print(df.describe(include='all'))
    print("\nDataFrame Info:")
    print(df.info())
    print("\nMissing Values:")
    print(df.isnull().sum())

In [5]:
def retry_payment_request(session, url, method= "POST", max_retries=5, retry_delay=1, **kwargs):
    """
    Retry a session.post request if it gets 402 Payment Required.

    The first 402 triggers the payment, subsequent requests should succeed.
    """
    for attempt in range(max_retries):
        response = session.request(method, url, **kwargs)

        if response.status_code == 402:
            if attempt < max_retries - 1:
                print(f"[RETRY] Got 402, retrying... (attempt {attempt + 1}/{max_retries})")
                time.sleep(retry_delay)
                continue
            else:
                print(f"[RETRY] ✗ Failed after {max_retries} attempts (402 Payment Required)")
                raise Exception(f"Payment failed after {max_retries} retries")

        # Success or other error
        return response

    return response

In [6]:
def get_regime():
    url = f"{BASE_URL}/regime"
    response = retry_payment_request(session, url, "GET")
    response.raise_for_status()
    return response.json()

def get_news():
    url = f"{BASE_URL}/news"
    response = retry_payment_request(session, url, "GET")
    response.raise_for_status()
    return response.json()

def post_advise(wallet_address: str, network: str):
    url = f"{BASE_URL}/advise"
    payload = {"wallet_address": wallet_address, "network": network}
    response = retry_payment_request(session, url, "POST", json=payload)
    response.raise_for_status()
    return response.json()

def post_portfolio(wallet_address: str, network: str, include_metrics: Optional[bool] = False):
    url = f"{BASE_URL}/portfolio"
    payload = {"wallet_address": wallet_address, "network": network, "include_metrics": include_metrics}
    response = retry_payment_request(session, url, "POST", json=payload)
    response.raise_for_status()
    return response.json()

def post_signal(network: str):
    url = f"{BASE_URL}/signal"
    payload = {"network": network}
    response = retry_payment_request(session, url, "POST", json=payload)
    response.raise_for_status()
    return response.json()

def get_model_metrics():
    url = f"{BASE_URL}/model/metrics"
    response = retry_payment_request(session, url, "GET")
    response.raise_for_status()
    return response.json()

In [7]:
def make_chat_request(session_id: Optional[str],
                      message: str,
                      wallet_address: Optional[str],
                      network: Optional[str]):
    url = f"{BASE_URL}/chat"
    payload = {
        "session_id": session_id,
        "message": message,
        "wallet_address": wallet_address,
        "network": network
    }

    response = retry_payment_request(session, url, json=payload)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code} - {response.text}")
        response.raise_for_status()

In [8]:
def get_wallet_historical(address: str, network: str):
    url = f"{BASE_URL}/wallet/historical"
    response = retry_payment_request(session, url,json={"wallet_address":address,"network":network})
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

In [9]:
def get_wallet_performance(address: str, network: str):
    url = f"{BASE_URL}/wallet/performance"
    response = retry_payment_request(session, url,json={"wallet_address":address,"network":network})
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

# /advise

In [57]:
resp = post_advise(WALLET_ADDRESS, NETWORK)
print(json.dumps(resp, indent=2))

KeyboardInterrupt: 

# /regime

In [ ]:
resp = get_regime()
print(json.dumps(resp, indent=2))

[RETRY] Got 402, retrying... (attempt 1/5)
{
  "regime": "Risk-Off",
  "confidence": 0.9999997058166289,
  "risk_on_probability": 2.941833710576118e-07,
  "date": "2025-11-25 00:00:00",
  "features": {
    "btc_price": 87341.890625,
    "btc_returns_30d": -0.23700511169684457,
    "btc_rsi": 20.91414776827625,
    "btc_drawdown": -0.2998788100742445,
    "fed_funds": 4.09,
    "cpi_yoy": 0.02502456951989096,
    "vix": 18.56,
    "yield_curve_spread": 0.5799999999999996
  },
  "dkg_ual": "did:dkg:otp:20430/0xcdb28e93ed340ec10a71bba00a31dbfcf1bd5d37/405836",
  "dkg_network": "otp:20430"
}


# /portfolio

In [ ]:
resp = post_portfolio(WALLET_ADDRESS, NETWORK, include_metrics=True)
print(json.dumps(resp, indent=2))

[RETRY] Got 402, retrying... (attempt 1/5)
{
  "wallet": "0x27238ff1Bf4Ef450AF1214a95963B9a92b351f08",
  "network": "opt-mainnet",
  "total_value_usd": 0.2431641275050095,
  "composition": {
    "ETH": 98.7,
    "OP": 1.3,
    "VELO": 0.0,
    "Visit https://op-reward.com to claim rewards": 0.0,
    "Airdrop(s) to be claimed at https://t.ly/drops": 0.0
  },
  "holdings": [
    {
      "symbol": "ETH",
      "name": "Ethereum",
      "balance": 7.8957777675907e-05,
      "price_usd": 3039.5755901293,
      "value_usd": 0.2399981336745431,
      "weight": 0.9869800127882713
    },
    {
      "symbol": "OP",
      "name": "Optimism",
      "balance": 0.009407180417239145,
      "price_usd": 0.3329214664,
      "value_usd": 0.00313185229919662,
      "weight": 0.012879581915848586
    },
    {
      "symbol": "VELO",
      "name": "VelodromeV2",
      "balance": 0.001093492453213432,
      "price_usd": 0.0312224663,
      "value_usd": 3.414153126976071e-05,
      "weight": 0.0001404052958

# /news

In [ ]:
resp = get_news()
print(json.dumps(resp, indent=2))

[RETRY] Got 402, retrying... (attempt 1/5)
{
  "generated_at": "2025-11-28T19:09:48.093649",
  "analysis": "### Smart Money Token Flow Analysis\n\n#### 1. Token Accumulation and Selling\n**Accumulating Tokens:**\n- **BIAO (BmQ3o2V3g2qJ46SQKM24fZtrcjc1cc4WXwbHvMd9pump, Solana)**: Net flow of $12,583.36 over 24 hours, with a market cap of $475,913.\n- **RAIL (0xe76c6c83af64e4c60245d8c7de953df673a7a33d, Ethereum)**: Positive net flow of $11,866.47 in 24 hours, despite a negative trend over the past week and month.\n- **LSETH (0x8c1bed5b9a0928467c9b1341da1d7bd5e10b6549, Ethereum)**: Gained $6,545.60 in 24 hours, continuing a positive trend over the past week and month.\n- **OLAF (7biCCUthUHDn48jTpYKFExvpJzxTcWoXjwVDBCyZpump, Solana)**: Accumulated $4,244.79 in 24 hours, with a market cap of $94,461.\n- **SPONGEBOB (7L3PEpzNP2ZkoVVuG1gvYCUu5yTf1LFgJkAi3CxNpump, Solana)**: Net flow of $2,546.87 in 24 hours.\n\n**Selling Tokens:**\n- **RAIL (0xe76c6c83af64e4c60245d8c7de953df673a7a33d, Ethereu

# /wallet/historical

In [ ]:
result = get_wallet_historical(WALLET_ADDRESS, NETWORK)
print(json.dumps(result, indent=4))

[RETRY] Got 402, retrying... (attempt 1/5)
{
    "wallet_address": "0x27238ff1bf4ef450af1214a95963b9a92b351f08",
    "start_date": "2023-07-26",
    "end_date": "2025-11-29",
    "returns": [
        {
            "date": "2023-07-26",
            "portfolio_daily_return": 0.0,
            "cumulative_return_pct": 0.0,
            "total_value_usd": 7.735202584246118
        },
        {
            "date": "2023-07-27",
            "portfolio_daily_return": 0.010770587311215862,
            "cumulative_return_pct": 1.0828798889979785,
            "total_value_usd": 7.818543563915914
        },
        {
            "date": "2023-07-28",
            "portfolio_daily_return": -0.007298806973170896,
            "cumulative_return_pct": 0.3477813947838193,
            "total_value_usd": 6.411046417505096
        },
        {
            "date": "2023-07-29",
            "portfolio_daily_return": 0.00619913191451519,
            "cumulative_return_pct": 0.9717826637464899,
            "tot

In [ ]:
result.keys()

dict_keys(['wallet_address', 'start_date', 'end_date', 'returns', 'composition', 'daily_balances'])

In [ ]:
df_returns = pd.DataFrame(result['returns'])

describe_df(df_returns)

DataFrame Description:
              date  portfolio_daily_return  cumulative_return_pct  \
count          858              858.000000             858.000000   
unique         858                     NaN                    NaN   
top     2023-07-26                     NaN                    NaN   
freq             1                     NaN                    NaN   
mean           NaN                0.000466              41.748428   
std            NaN                0.036289              40.322875   
min            NaN               -0.158733             -27.017704   
25%            NaN               -0.017596              10.913151   
50%            NaN                0.000773              40.748520   
75%            NaN                0.018216              68.131156   
max            NaN                0.193684             138.671952   

        total_value_usd  
count        858.000000  
unique              NaN  
top                 NaN  
freq                NaN  
mean           1.4

In [ ]:
df_returns = df_returns.set_index('date')

In [ ]:
cm = ChartMaker(shuffle_colors=True)
cm.build(
    df = df_returns,
    axes_data=dict(y1=['cumulative_return_pct']),
    title='cumulative_return_pct',
    chart_type={'y1':'line'},
    options = {'margin':dict(t=100),'annotations':True,
                'ticksuffix':{'y1':'%'},'auto_title':False,
                'show_legend':False,'legend_placement':{'x':0.01,'y':1.05}},
)
cm.add_title(
    title='Cumulative Return Percentage Over Time',
    subtitle=f'Daily through {df_returns.index.max()}',
    x=0.2,
    y=0.9
)
cm.add_annotations()
cm.show_fig()
cm.save_fig(save_directory='../img')

Saving figure to: ../img\cumulative_return_pct.png


In [ ]:
cm = ChartMaker(shuffle_colors=True)
cm.build(
    df = df_returns,
    axes_data=dict(y1=['total_value_usd']),
    title='total_value_usd',
    chart_type={'y1':'line'},
    options = {'margin':dict(t=100),'annotations':True,
                'tickprefix':{'y1':'$'},'auto_title':False,
                'show_legend':False,'legend_placement':{'x':0.01,'y':1.05}},
)
cm.add_title(
    title='Total Value USD Over Time',
    subtitle=f'Daily through {df_returns.index.max()}',
    x=0.2,
    y=0.9
)
cm.add_annotations()
cm.show_fig()
cm.save_fig(save_directory='../img')

Saving figure to: ../img\total_value_usd.png


In [ ]:
df_composition = pd.DataFrame(result['composition']['weights'], index=pd.to_datetime(result['composition']['dates']))

describe_df(df_composition)

DataFrame Description:
       ETH_weight   OP_weight   USDT_weight  VELO_weight
count  858.000000  858.000000  8.580000e+02   858.000000
mean     0.848628    0.126583  1.971295e-02     0.005077
std      0.277509    0.233217  4.147424e-02     0.041192
min      0.188729    0.000364  0.000000e+00     0.000002
25%      0.904532    0.010205  4.554783e-15     0.000033
50%      0.984144    0.015416  3.422236e-14     0.000111
75%      0.989752    0.082152  1.034614e-02     0.000520
max      0.999634    0.717733  1.430552e-01     0.430057

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 858 entries, 2023-07-26 to 2025-11-29
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ETH_weight   858 non-null    float64
 1   OP_weight    858 non-null    float64
 2   USDT_weight  858 non-null    float64
 3   VELO_weight  858 non-null    float64
dtypes: float64(4)
memory usage: 33.5 KB
None

Missing Values:
ETH_weigh

In [ ]:
df_composition

,ETH_weight,OP_weight,USDT_weight,VELO_weight
2023-07-26,0.907541,0.091924,0.000000e+00,0.000535
2023-07-27,0.904604,0.094786,0.000000e+00,0.000611
2023-07-28,0.879158,0.120660,0.000000e+00,0.000182
2023-07-29,0.881227,0.118558,0.000000e+00,0.000215
2023-07-30,0.879401,0.120379,0.000000e+00,0.000220
...,...,...,...,...
2025-11-25,0.987157,0.012699,6.766360e-14,0.000144
2025-11-26,0.986955,0.012901,6.746815e-14,0.000143
2025-11-27,0.987235,0.012622,6.587742e-14,0.000144
2025-11-28,0.986842,0.013013,6.619996e-14,0.000145


In [ ]:
cm = ChartMaker(shuffle_colors=True)
cm.build(
    df = df_composition * 100,
    axes_data=dict(y1=df_composition.columns.tolist()),
    title='composition',
    chart_type={'y1':'area'},
    options = {'margin':dict(t=100),'annotations':True,
                'ticksuffix':{'y1':'%'},'auto_title':False,
                'show_legend':False,'legend_placement':{'x':0.01,'y':1.05}},
)
cm.add_title(
    title='Composition Over Time',
    subtitle=f'Daily through {df_composition.index.max()}',
    x=0.2,
    y=0.9
)
cm.add_annotations()
cm.show_fig()
cm.save_fig(save_directory='../img')

Saving figure to: ../img\composition.png


In [ ]:
# Flatten daily_balances into a list of records
records = []
for date, data in result['daily_balances'].items():
    for token in data['tokens']:
        records.append({
            'date': date,
            'total_value_usd': data['total_value_usd'],        
            **token
        })

df_balances = pd.DataFrame(records)
df_balances['date'] = pd.to_datetime(df_balances['date'])      
df_balances = df_balances.sort_values('date')

df_totals = pd.DataFrame([
    {'date': date, 'total_value_usd':
data['total_value_usd']}
    for date, data in result['daily_balances'].items()
])
df_totals['date'] = pd.to_datetime(df_totals['date'])
df_totals = df_totals.sort_values('date').set_index('date')

In [ ]:
describe_df(df_balances)

DataFrame Description:
                                 date  total_value_usd symbol      balance  \
count                            3432      3432.000000   3432  3432.000000   
unique                            NaN              NaN      4          NaN   
top                               NaN              NaN   VELO          NaN   
freq                              NaN              NaN    858          NaN   
mean    2024-09-26 11:59:59.999999744         1.451740    NaN     2.280474   
min               2023-07-26 00:00:00         0.019270    NaN     0.000000   
25%               2024-02-25 00:00:00         0.259519    NaN     0.000773   
50%               2024-09-26 12:00:00         0.768087    NaN     0.001093   
75%               2025-04-29 00:00:00         1.973225    NaN     0.009407   
max               2025-11-29 00:00:00        24.308505    NaN   459.922723   
std                               NaN         2.526534    NaN    30.761131   

          price_usd    value_usd  
count

In [ ]:
df_balances = df_balances.set_index('date')

In [ ]:
cm = ChartMaker(shuffle_colors=True)
cm.build(
    df = df_balances[df_balances.index >= '2023-01-01'],
    groupby_col='symbol',
    num_col='value_usd',
    title='TVL by Token',
    chart_type={'y1':'area'},
    options = {
        'margin':dict(t=125),
        'annotations':False,
        'tickformat': dict(x='%b<br>%y'),
        'tickprefix':{'y1':'$'},
        'decimals': True,
        'barmode':'relative',
        'legend_placement':dict(x=0.1,y=0.95),
        "dimensions": dict(width=1000, height=600),
    },
)
cm.add_title(
    subtitle=f'Daily through {df_balances.index.max()}',
    x=0.2,
    y=0.9
)
# cm.add_annotations()
cm.show_fig()

groupby_col and num_col passed...


In [ ]:
describe_df(df_totals)

DataFrame Description:
       total_value_usd
count       858.000000
mean          1.451740
std           2.527640
min           0.019270
25%           0.259624
50%           0.768087
75%           1.971758
max          24.308505

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 858 entries, 2023-07-26 to 2025-11-29
Data columns (total 1 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   total_value_usd  858 non-null    float64
dtypes: float64(1)
memory usage: 13.4 KB
None

Missing Values:
total_value_usd    0
dtype: int64


In [ ]:
cm = ChartMaker(shuffle_colors=True)
cm.build(
    df = df_totals[df_totals.index >= '2023-01-01'],
    axes_data=dict(y1=["total_value_usd"]),
    title='composition',
    chart_type={'y1':'area'},
    options = {'margin':dict(t=100),'annotations':True,
                'tickprefix':{'y1':'$'},'auto_title':False,
                'show_legend':False,'legend_placement':{'x':0.01,'y':1.05}},
)
cm.add_title(
    title='Total Value Over Time',
    subtitle=f'Daily through {df_totals.index.max()}',
    x=0.2,
    y=0.9
)
cm.add_annotations()
cm.show_fig()
cm.save_fig(save_directory='../img')

Saving figure to: ../img\composition.png


# /wallet/performance

In [ ]:
result = get_wallet_performance(WALLET_ADDRESS, NETWORK)
print(json.dumps(result, indent=4))

[RETRY] Got 402, retrying... (attempt 1/5)
{
    "wallet_address": "0x27238ff1bf4ef450af1214a95963b9a92b351f08",
    "total_return": 0.3744226157148913,
    "cagr": -0.7713285357013951,
    "sharpe_ratio": 0.12449992001308524,
    "sortino_ratio": 0.1256492858412622,
    "calmar_ratio": -1.1459686410762027,
    "volatility": 0.6947847259151936,
    "var_95": -0.05783502437838186,
    "cvar_95": -0.08265808709521065,
    "max_drawdown": -0.6730799675085565,
    "beta": 1.1389538942880246,
    "alpha": -0.5564129254156466,
    "treynor_ratio": 0.08329872352063385,
    "information_ratio": -0.7632307079704537,
    "start_value": 7.735202584246118,
    "end_value": 0.24264218842236848,
    "start_date": "2023-07-26",
    "end_date": "2025-11-29",
    "days": 857,
    "gas_spent_eth": 0.0
}


# /model/historical

In [ ]:
import requests

r = requests.get(f"{BASE_URL}/model/historical")

In [ ]:
data = r.json()

In [ ]:
data.keys()

dict_keys(['start_date', 'end_date', 'initial_capital', 'strategies', 'cumulative_returns', 'live_agent'])

In [ ]:
for key in data['strategies']:
    print(f'{key}: {data["strategies"][key]}')

buy_and_hold: {'cagr': 0.3167, 'sharpe_ratio': 0.7951, 'sortino_ratio': 1.2271, 'max_drawdown': -0.3056, 'calmar_ratio': 1.0363, 'var_95': -0.036, 'cvar_95': -0.0494, 'volatility': 0.444, 'cumulative_return': 0.368, 'final_value': 13494.87}
always_risk_on: {'cagr': 0.3167, 'sharpe_ratio': 0.7951, 'sortino_ratio': 1.2271, 'max_drawdown': -0.3056, 'calmar_ratio': 1.0363, 'var_95': -0.036, 'cvar_95': -0.0494, 'volatility': 0.444, 'cumulative_return': 0.368, 'final_value': 13494.87}
always_risk_off: {'cagr': 0.0437, 'sharpe_ratio': 247.403, 'sortino_ratio': 0.0, 'max_drawdown': 0.0, 'calmar_ratio': 0.0, 'var_95': 0.0001, 'cvar_95': 0.0001, 'volatility': 0.0001, 'cumulative_return': 0.0499, 'final_value': 10498.79}
model: {'cagr': 0.5767, 'sharpe_ratio': 1.5014, 'sortino_ratio': 1.8769, 'max_drawdown': -0.2252, 'calmar_ratio': 2.5607, 'var_95': -0.024, 'cvar_95': -0.0371, 'volatility': 0.3247, 'cumulative_return': 0.6797, 'final_value': 16570.01}


In [ ]:
model_returns = pd.DataFrame(data['cumulative_returns'])

describe_df(model_returns)

DataFrame Description:
                       date  buy_and_hold  always_risk_on  always_risk_off  \
count                   417    417.000000      417.000000       417.000000   
unique                  417           NaN             NaN              NaN   
top     2024-09-30 00:00:00           NaN             NaN              NaN   
freq                      1           NaN             NaN              NaN   
mean                    NaN      1.559921        1.559921         1.025241   
std                     NaN      0.236003        0.236003         0.014396   
min                     NaN      0.951800        0.951800         1.000000   
25%                     NaN      1.431000        1.431000         1.012900   
50%                     NaN      1.599600        1.599600         1.025200   
75%                     NaN      1.737800        1.737800         1.037700   
max                     NaN      1.969900        1.969900         1.049900   

             model  
count   417.000000 

In [ ]:
model_returns = model_returns.set_index('date')

In [ ]:
cm = ChartMaker(shuffle_colors=True)
cm.build(
    df = model_returns * 100,
    axes_data=dict(y1=model_returns.columns.tolist()),
    title='normalized_returns',
    chart_type={'y1':'line'},
    options = {'margin':dict(t=100),'annotations':True,
                'tickprefix':{'y1':'$'},'auto_title':False,
                'show_legend':True,'legend_placement':{'x':0.01,'y':1.05}},
)
cm.add_title(
    title='Normalized Returns Over Time',
    subtitle=f'Daily through {model_returns.index.max()}',
    x=0.2,
    y=0.9
)
cm.add_annotations()
cm.show_fig()
cm.save_fig(save_directory='../img')

Saving figure to: ../img\normalized_returns.png


In [ ]:
pd.DataFrame(data['live_agent']['decisions'])

,timestamp,date,risk_on,confidence,btc_price,btc_allocation,cash_allocation,btc_holdings,cash_holdings,portfolio_value,rebalanced,fee_paid,features
0,2025-11-25T01:07:20.435868,2025-11-20 00:00:00,False,1.0,86631.898438,0.0,1.0,0.0,10000.0,10000.0,False,0,"{'m2': 22212.5, 'gdp': 30485.729, 'cpi': 324.3..."
1,2025-11-27T05:00:00.162891,2025-11-25 00:00:00,False,1.0,87341.890625,0.0,1.0,0.0,10000.0,10000.0,False,0,"{'m2': 22298.1, 'gdp': 30485.729, 'cpi': 324.3..."


In [ ]:
for i in data['live_agent']['decisions']:
    print(i.get("features"))

{'m2': 22212.5, 'gdp': 30485.729, 'cpi': 324.368, 'fed_funds': 4.09, 'recession': 0.0, 'dxy': 121.363, 'unemployment': 4.4, 'vix': 26.42, 'treasury_10y': 4.1, 'treasury_2y': 3.55, 'm2_yoy': 0.037584256279223904, 'm2_mom': 0.0, 'gdp_growth': 0.0, 'cpi_yoy': 0.02502456951989096, 'cpi_mom': 0.0, 'fed_funds_change': 0.0, 'fed_funds_ma_30': 4.09, 'yield_curve_spread': 0.5499999999999998, 'yield_curve_inverted': 0.0, 'vix_ma_30': 18.777333333333335, 'vix_elevated': 1.0, 'dxy_change': 0.0, 'dxy_ma_30': 121.53656333333333, 'unemployment_change': 0.0, 'btc_price': 86631.8984375, 'btc_returns_1d': -0.05285126891851111, 'btc_returns_7d': -0.1310523811915718, 'btc_returns_30d': -0.20137922521228235, 'btc_returns_60d': -0.24867892389685609, 'btc_rsi': 25.755715673550043, 'btc_drawdown': -0.30557001473667494, 'btc_volatility_30d': 0.022740900022393984, 'btc_price_vs_ma_30': -0.16484007763948677, 'btc_price_vs_ma_200': -0.21508241709722506, 'btc_ma_30_above_200': 0.0}
{'m2': 22298.1, 'gdp': 30485.729

# /model/metrics

In [ ]:
resp = get_model_metrics()
print(json.dumps(resp, indent=2))

{
  "model_type": "Logistic Regression",
  "training_date": "2025-11-22 15:46:47",
  "total_features": 35,
  "train_metrics": {
    "accuracy": 0.982,
    "precision": 1.0,
    "recall": 0.9733,
    "f1_score": 0.9865,
    "roc_auc": 0.9989,
    "confusion_matrix": [
      [
        108,
        0
      ],
      [
        6,
        219
      ]
    ]
  },
  "test_metrics": {
    "accuracy": 0.9167,
    "precision": 1.0,
    "recall": 0.6667,
    "f1_score": 0.8,
    "roc_auc": 0.9977,
    "confusion_matrix": [
      [
        63,
        0
      ],
      [
        7,
        14
      ]
    ]
  },
  "cross_validation": {
    "n_folds": 4,
    "n_splits": 5,
    "fold_results": [
      {
        "fold": 2,
        "train_size": 141,
        "test_size": 69,
        "accuracy": 0.8695652173913043,
        "precision": 1.0,
        "recall": 0.4,
        "f1": 0.5714285714285714,
        "roc_auc": 0.9617283950617284
      },
      {
        "fold": 3,
        "train_size": 210,
        "t

# /chat

In [ ]:
init_resp = make_chat_request(None, "describe my wallet's performance", WALLET_ADDRESS, NETWORK)
session_id = init_resp['session_id']
print(session_id)

6a22ed2d-0854-4f9f-a6f0-c875d1917883


In [ ]:
print(json.dumps(init_resp, indent=2))

{
  "session_id": "6a22ed2d-0854-4f9f-a6f0-c875d1917883",
  "response": "{'response': \"Your wallet's performance can be analyzed based on several key metrics over the 856 days:\\n\\n1. **Total Return**: Your portfolio has achieved a total return of 36.7%. This indicates that, overall, your investments have grown in value since inception.\\n\\n2. **Sharpe Ratio**: At 0.12, this ratio suggests that your portfolio's risk-adjusted returns are relatively low. A Sharpe ratio below 1 indicates that the returns may not justify the level of risk taken.\\n\\n3. **Sortino Ratio**: Also at 0.12, this ratio focuses on downside risk. Similar to the Sharpe ratio, it indicates that your portfolio's performance is not compensating adequately for the risks associated with negative returns.\\n\\n4. **Max Drawdown**: Your portfolio has experienced a maximum drawdown of -67.3%. This significant decline indicates a period of substantial loss from the peak value, highlighting considerable volatility and ris

In [ ]:
resp = make_chat_request(session_id=session_id, 
                         message="Should I consider swapping into RWA tokens, yield-accuring or reward bearing tokens, like susde?.", 
                         wallet_address=None,                
                         network=None)

[RETRY] Got 402, retrying... (attempt 1/5)


In [ ]:
print(json.dumps(resp, indent=2))

{
  "session_id": "6a22ed2d-0854-4f9f-a6f0-c875d1917883",
  "response": "{'response': \"Given the current macro regime characterized as Risk-Off, it is prudent to approach investment decisions with caution. Here are some insights regarding your consideration of swapping into Real World Asset (RWA) tokens or yield-earning/reward-bearing tokens like Susde:\\n\\n1. **Market Conditions**: In a Risk-Off environment, investors typically seek safety and stability. RWA tokens can offer a hedge against volatility because they are often backed by physical assets, potentially providing more stability compared to traditional cryptocurrencies.\\n\\n2. **Yield-Generating Tokens**: Tokens that provide yield or rewards can be attractive; however, they may also carry risks associated with the underlying assets and market conditions. While they can offer passive income, it is essential to evaluate the sustainability of the yield and the creditworthiness of the underlying projects.\\n\\n3. **Portfolio Di

# /signal

In [ ]:
resp = post_signal("base-sepolia")

[RETRY] Got 402, retrying... (attempt 1/5)


In [ ]:
print(json.dumps(resp, indent=2))

{
  "risk_on": false,
  "confidence": 99,
  "last_updated": 1764028800,
  "transaction_hash": null,
  "backend_submitted": false,
  "signature": "0xe9d35d94205c5af7db1de85e9c393eff7dec5c0b4d6582107bc9de4c0b1f13e97f3017cccec8b24e20577a78a4cf22a46ee122cc9fc71c33faf4348b866e05731b",
  "snapshot_hash": "0xdfed80af9d82b93543039f144bfa4692a5328eed541b340d01ed159a4403e1c3",
  "signer_address": "0x85bADB8118f98b6AD8eaB5880c8829AF8adA44cF",
  "error": "Transaction submission returned None"
}


# /wallet/live

In [10]:
client = X402WebSocketClient(
    base_url=BASE_URL,
    wallet_address=WALLET_ADDRESS,
    private_key=PRIVATE_KEY,
    network=NETWORK,
)

[x402] Account: 0xeb1d7c1b6cCc8b48bC764A9C0c5d68E081F0aBEF


In [11]:
try:
    await client.connect()
    await client.stream()
except KeyboardInterrupt:
    print("\n⏹️  Interrupted")
except Exception as e:
    print(f"\n❌ {e}")
    import traceback
    traceback.print_exc()
finally:
    await client.disconnect()

1. Making x402 payment via HTTP...
   ✓ Payment successful
2. Connecting to WebSocket...
3. Connected!
   Session: 8a3558be-d2a4-4170-ae89-a6ab5d704d2b
   Expires: 2025-11-29T17:00:55.546400+00:00

Streaming portfolio updates...
------------------------------------------------------------
[2025-11-29T16:55:56] #1 💰 $0.24
   ETH: $0.24 (98.7%)
   OP: $0.00 (1.3%)
   VELO: $0.00 (0.0%)
   Visit https://op-reward.com to claim rewards: $0.00 (0.0%)
   Airdrop(s) to be claimed at https://t.ly/drops: $0.00 (0.0%)
[2025-11-29T16:56:26] #2 💰 $0.24 📈 +0.00%
   ETH: $0.24 (98.7%)
   OP: $0.00 (1.3%)
   VELO: $0.00 (0.0%)
   Visit https://op-reward.com to claim rewards: $0.00 (0.0%)
   Airdrop(s) to be claimed at https://t.ly/drops: $0.00 (0.0%)
[2025-11-29T16:56:58] #3 💰 $0.24 📉 -0.00%
   ETH: $0.24 (98.7%)
   OP: $0.00 (1.3%)
   VELO: $0.00 (0.0%)
   Visit https://op-reward.com to claim rewards: $0.00 (0.0%)
   Airdrop(s) to be claimed at https://t.ly/drops: $0.00 (0.0%)
[2025-11-29T16:57:29] #

CancelledError: 